In [1]:
import geopandas as gpd
import pandas as pd
import fiona as fi
import glob
from shapely.geometry import Point, LineString, Polygon
import numpy
import random

random.seed(0)

In [2]:
#Part 1

shed_files = fi.listlayers(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week4\lab3.gpkg')
sheds = []
for files in shed_files:
    if "wdbhuc" in files:
        sheds.append(files)

#create empty dictionary to append generated points     
sample_points = {'point_id': [], 'geometry':[], 'HUC': []}
for polys in sheds:
    shed_files_gdf = gpd.read_file(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week4\lab3.gpkg', layer = polys)
    huccode = [f for f in shed_files_gdf.columns if 'HUC' in f][0]
    for idx, row in shed_files_gdf.iterrows():
        j = 0
        #get extents of each polygon, convert to square kilometers, calculate number of points, n
        extent = row['geometry'].bounds
        area_km = row["Shape_Area"]/1000000
        n = (int(round(area_km*0.05)))
        #generate random points within our extents
        while j < n:
            x = random.uniform(extent[0], extent[2])
            y = random.uniform(extent[1], extent[3])
            p = (Point(x,y))
            #if the random point generated is within the polygon, append it to the empty dictionary
            if row['geometry'].contains(p):
                sample_points['geometry'].append(p)
                sample_points['point_id'].append(row[huccode][:8])
                sample_points['HUC'].append(huccode)
                j += 1

In [4]:
#Part 2

#create geodataframe, perform join, group and get average, print
sample_points_gdf = gpd.GeoDataFrame(sample_points)
ssurgo = gpd.read_file(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\week4\lab3.gpkg', layer = 'ssurgo_mapunits_lab3')
sample_points_gdf = sample_points_gdf.set_crs("EPSG:26913")
new_class = gpd.sjoin(sample_points_gdf, ssurgo,  how='left', op="within")

group = new_class.groupby(by=['HUC', 'point_id']).mean()
print(group['aws0150'])

HUC    point_id
HUC12  10190005    10.841360
       10190006    10.492456
       10190007    10.623682
HUC8   10190005    11.473190
       10190006     7.877788
       10190007    11.386864
Name: aws0150, dtype: float64
